In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn

In [10]:
traindata = pd.read_csv('datasets/upvotes/train_NIR5Yl1.csv')
traindata.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [11]:
testdata = pd.read_csv('datasets/upvotes/test_8i3B3FC.csv')
testdata.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [12]:
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141448 entries, 0 to 141447
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          141448 non-null  int64  
 1   Tag         141448 non-null  object 
 2   Reputation  141448 non-null  float64
 3   Answers     141448 non-null  float64
 4   Username    141448 non-null  int64  
 5   Views       141448 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 6.5+ MB


## Clean Data

In [13]:
testdata =testdata.dropna()
testdata.describe()

,ID,Reputation,Answers,Username,Views
count,141448.000000,1.414480e+05,141448.000000,141448.000000,1.414480e+05
mean,235743.073497,7.920927e+03,3.914873,81348.231117,2.984633e+04
std,136269.867118,2.791072e+04,3.577460,49046.098215,8.034374e+04
min,7.000000,0.000000e+00,0.000000,4.000000,9.000000e+00
25%,117797.000000,2.860000e+02,2.000000,40222.750000,2.608000e+03
50%,235830.000000,1.245000e+03,3.000000,78795.500000,8.977000e+03
75%,353616.000000,5.123000e+03,5.000000,122149.000000,2.698925e+04
max,471488.000000,1.042428e+06,73.000000,175737.000000,5.004669e+06


## Ordinal Encoder

In [14]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [15]:
tag_cat = traindata[['Tag']]
tag_cat_encoded = ordinal_encoder.fit_transform(tag_cat)
tag_cat_encoded[:10]

array([[0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [7.],
       [1.],
       [4.],
       [4.],
       [1.]])

## One-Hot Encoder

In [16]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
tag_cat_1hot = cat_encoder.fit_transform(tag_cat)
tag_cat_1hot.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Custom Transformer

In [17]:
def transform(X):
    answer_index, views_index = 3,5
    answers_per_view = X[:, answer_index] / X[:, views_index]
    return np.c_[X, answers_per_view]


In [18]:
extra_cols = transform(traindata.values)

In [19]:
training_extra = pd.DataFrame(
    extra_cols,
    columns=list(traindata.columns)+["answers per view"],
    index=traindata.index)
training_extra.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,answers per view
0,52664,a,3942.0,2.0,155623,7855.0,42.0,0.000255
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0.000215
2,468453,c,1358.0,4.0,56177,8067.0,60.0,0.000496
3,96996,a,264.0,3.0,168793,27064.0,9.0,0.000111
4,131465,c,4271.0,4.0,112223,13986.0,83.0,0.000286
